In [174]:
import pandas as pd
import numpy as np
import soccerdata as sd 
import requests
import json
import itertools
import mezzala
import fotmob_scraper as fms
import pandas_montecarlo

In [46]:
mh = pd.read_csv('data/super_league_matches_historical.csv')

In [47]:
mh['Home'].unique()

array(['Servette', 'Thun', 'Grasshoppers', 'Luzern', 'St. Gallen',
       'Basel', 'Lausanne', 'Sion', 'Young Boys', 'Zurich', 'Aarau',
       'Vaduz', 'Lugano', 'Xamax'], dtype=object)

In [48]:
url = 'https://raw.githubusercontent.com/openfootball/football.json/master/2020-21/ch.1.json'


In [49]:
from urllib import response


response = requests.get(url).json()

data = response['matches']
data[0:3]

[{'round': 'Spieltag 1',
  'date': '2020-09-19',
  'team1': 'FC Lugano',
  'team2': 'FC Luzern',
  'score': {'ft': [2, 1]}},
 {'round': 'Spieltag 1',
  'date': '2020-09-19',
  'team1': 'BSC Young Boys',
  'team2': 'FC Zürich',
  'score': {'ft': [2, 1]}},
 {'round': 'Spieltag 1',
  'date': '2020-09-20',
  'team1': 'FC Basel',
  'team2': 'FC Vaduz',
  'score': {'ft': [2, 2]}}]

In [50]:
mh

,Country,League,Season,Date,Time,Home,Away,HG,AG,Res,PH,PD,PA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA
0,Switzerland,Super League,2012/2013,13/07/2012,18:45,Servette,Basel,0,1,A,6.22,4.17,1.60,6.22,4.20,1.68,5.32,3.85,1.59
1,Switzerland,Super League,2012/2013,14/07/2012,18:45,Thun,Lausanne,0,0,D,1.75,3.80,5.23,1.77,3.81,5.38,1.70,3.51,4.85
2,Switzerland,Super League,2012/2013,15/07/2012,12:45,Grasshoppers,Sion,0,2,A,3.27,3.36,2.39,3.35,3.41,2.41,3.07,3.24,2.25
3,Switzerland,Super League,2012/2013,15/07/2012,12:45,Luzern,Zurich,1,1,D,2.04,3.65,3.89,2.20,3.80,3.89,2.10,3.33,3.31
4,Switzerland,Super League,2012/2013,15/07/2012,15:00,St. Gallen,Young Boys,1,1,D,3.62,3.67,2.12,3.69,3.67,2.15,3.32,3.43,2.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1802,Switzerland,Super League,2021/2022,22/05/2022,15:30,Sion,Servette,3,3,D,1.80,3.75,4.67,1.91,4.00,4.90,1.75,3.76,4.41
1803,Switzerland,Super League,2021/2022,22/05/2022,15:30,St. Gallen,Lausanne,4,0,H,1.41,5.34,7.12,1.50,5.90,7.50,1.38,5.35,6.47
1804,Switzerland,Super League,2021/2022,22/05/2022,15:30,Young Boys,Grasshoppers,3,0,H,1.31,6.66,7.75,1.35,7.00,9.50,1.28,6.17,8.13
1805,Switzerland,Super League,2021/2022,22/05/2022,15:30,Zurich,Luzern,2,3,A,2.32,4.10,2.80,2.34,4.20,2.91,2.25,3.97,2.71


In [51]:
mh_dict = mh.to_dict('records')

In [52]:
mh_dict[0]

{'Country': 'Switzerland',
 'League': 'Super League',
 'Season': '2012/2013',
 'Date': '13/07/2012',
 'Time': '18:45',
 'Home': 'Servette',
 'Away': 'Basel',
 'HG': 0,
 'AG': 1,
 'Res': 'A',
 'PH': 6.22,
 'PD': 4.17,
 'PA': 1.6,
 'MaxH': 6.22,
 'MaxD': 4.2,
 'MaxA': 1.68,
 'AvgH': 5.32,
 'AvgD': 3.85,
 'AvgA': 1.59}

In [53]:
adapter = mezzala.KeyAdapter(
    home_team='Home',
    away_team='Away',
    home_goals='HG',
    away_goals='AG'
)

In [54]:
model = mezzala.DixonColes(adapter,
blocks=[
    mezzala.blocks.HomeAdvantage(),
    mezzala.blocks.TeamStrength(),
    mezzala.blocks.BaseRate(),
])
model.fit(mh_dict)

DixonColes(adapter=KeyAdapter(home_goals='HG', away_goals='AG', home_team='Home', away_team='Away'), blocks=[TeamStrength(), HomeAdvantage(), BaseRate()]), weight=UniformWeight()

In [55]:
match_to_predict = {
    'Home': 'Basel',
    'Away': 'Wintherthur'
}

scorelines = model.predict_one(match_to_predict)

In [56]:
scorelines

[ScorelinePrediction(home_goals=0, away_goals=0, probability=0.046267132765495815),
 ScorelinePrediction(home_goals=0, away_goals=1, probability=0.032402528819888886),
 ScorelinePrediction(home_goals=0, away_goals=2, probability=0.016852939068046545),
 ScorelinePrediction(home_goals=0, away_goals=3, probability=0.005072441044509963),
 ScorelinePrediction(home_goals=0, away_goals=4, probability=0.0011450372860547435),
 ScorelinePrediction(home_goals=0, away_goals=5, probability=0.00020678176443267502),
 ScorelinePrediction(home_goals=0, away_goals=6, probability=3.111885425234821e-05),
 ScorelinePrediction(home_goals=0, away_goals=7, probability=4.0140998449773025e-06),
 ScorelinePrediction(home_goals=0, away_goals=8, probability=4.5306529461000517e-07),
 ScorelinePrediction(home_goals=0, away_goals=9, probability=4.545492009719379e-08),
 ScorelinePrediction(home_goals=0, away_goals=10, probability=4.10434170760906e-09),
 ScorelinePrediction(home_goals=0, away_goals=11, probability=3.36

In [57]:
mh['Home'].unique

<bound method Series.unique of 0           Servette
1               Thun
2       Grasshoppers
3             Luzern
4         St. Gallen
            ...     
1802            Sion
1803      St. Gallen
1804      Young Boys
1805          Zurich
1806          Luzern
Name: Home, Length: 1807, dtype: object>

In [58]:
mezzala.scorelines_to_outcomes(scorelines)

{Outcomes('Home win'): OutcomePrediction(outcome=Outcomes('Home win'), probability=0.6793841785547168),
 Outcomes('Draw'): OutcomePrediction(outcome=Outcomes('Draw'), probability=0.1917990810360604),
 Outcomes('Away win'): OutcomePrediction(outcome=Outcomes('Away win'), probability=0.12881674040922275)}

In [59]:
matches_to_predict = [{
    'Home': 'Basel',
    'Away': 'Zurich'
},{
    'Home': 'Zurich',
    'Away': 'Basel'
},]
scorelines = model.predict(matches_to_predict)

In [60]:
scorelines

[[ScorelinePrediction(home_goals=0, away_goals=0, probability=0.04331418778415269),
  ScorelinePrediction(home_goals=0, away_goals=1, probability=0.03411605240133336),
  ScorelinePrediction(home_goals=0, away_goals=2, probability=0.020078948990411037),
  ScorelinePrediction(home_goals=0, away_goals=3, probability=0.0068585911873164365),
  ScorelinePrediction(home_goals=0, away_goals=4, probability=0.0017570742782851586),
  ScorelinePrediction(home_goals=0, away_goals=5, probability=0.00036011010834069397),
  ScorelinePrediction(home_goals=0, away_goals=6, probability=6.150341874737962e-05),
  ScorelinePrediction(home_goals=0, away_goals=7, probability=9.003604564278271e-06),
  ScorelinePrediction(home_goals=0, away_goals=8, probability=1.1532982182259305e-06),
  ScorelinePrediction(home_goals=0, away_goals=9, probability=1.3131499063438104e-07),
  ScorelinePrediction(home_goals=0, away_goals=10, probability=1.345641902807186e-08),
  ScorelinePrediction(home_goals=0, away_goals=11, prob

In [61]:
final_score = scorelines[0][0]
for scores in scorelines[0]:
    if scores.probability > final_score.probability:
        final_score = scores

In [62]:
final_score.probability

0.0982344874193076

In [63]:
scorelines[0][0]

ScorelinePrediction(home_goals=0, away_goals=0, probability=0.04331418778415269)

In [64]:
df_schedule = fms.get_league_schedule(69)

getting fixtures for Super League


In [65]:
df_schedule

['id',
 'pageUrl',
 'displayTournament',
 'lnameArr',
 'color',
 'notStarted',
 'opponent.id',
 'opponent.name',
 'home.id',
 'home.name',
 'away.id',
 'away.name',
 'status.started',
 'status.cancelled',
 'status.finished',
 'status.startTimeStr',
 'status.startDateStr',
 'status.startDateStrShort']

In [66]:
league_url = "https://www.fotmob.com/api/leagues?id={}"
league_id = 69
response = requests.get(league_url.format(league_id)).json()
df_fixtures = pd.json_normalize(response, record_path=["matches"])

In [67]:
df_fixtures_short = df_fixtures[['id','home.name','away.name', 'home.id', 'away.id','status.startDateStr']]

In [68]:
df_fixtures_short['home.name'].unique()

array(['Young Boys', 'Winterthur', 'Lugano', 'Servette', 'FC Zürich',
       'St. Gallen', 'Basel', 'Grasshopper', 'Sion', 'Luzern'],
      dtype=object)

In [69]:
mh["Home"].unique()

array(['Servette', 'Thun', 'Grasshoppers', 'Luzern', 'St. Gallen',
       'Basel', 'Lausanne', 'Sion', 'Young Boys', 'Zurich', 'Aarau',
       'Vaduz', 'Lugano', 'Xamax'], dtype=object)

In [70]:
dict_teams = {
    'Young Boys':'Young Boys', 
    'Thun':'Winterthur', 
    'Lugano':'Lugano', 
    'Servette':'Servette', 
    'FC Zürich':'Zurich',
    'St. Gallen':'St. Gallen', 
    'Basel':'Basel', 
    'Grasshoppers':'Grasshopper', 
    'Sion':'Sion', 
    'Luzern':'Luzern',
    'Lausanne':'Winterthur',
    'Aarau':'Winterthur',
    'Vaduz':'Winterthur',
    'Xamax':'Winterthur'
}

In [71]:
mh['Home'] = mh['Home'].map(dict_teams)
mh['Away'] = mh['Away'].map(dict_teams)

In [72]:
mh = mh[mh['Home'] != mh['Away']]

In [171]:
df_predictions = pd.DataFrame()

In [74]:
df_fixtures_short

,id,home.name,away.name,home.id,away.id,status.startDateStr
0,3905393,Young Boys,FC Zürich,10192,10243,"Sat, 16 Jul"
1,3905401,Winterthur,Basel,7894,9931,"Sat, 16 Jul"
2,3905400,Lugano,Sion,7896,10179,"Sun, 17 Jul"
3,3905399,Servette,St. Gallen,9777,10190,"Sun, 17 Jul"
4,3905397,FC Zürich,Luzern,10243,10199,"Sat, 23 Jul"
...,...,...,...,...,...,...
175,3905541,Basel,Grasshopper,9931,9956,29 May 2023
176,3905573,FC Zürich,Lugano,10243,7896,29 May 2023
177,3905540,Luzern,Servette,10199,9777,29 May 2023
178,3905528,St. Gallen,Sion,10190,10179,29 May 2023


In [75]:
mh_dict = mh.to_dict('records')

In [76]:
model = mezzala.DixonColes(adapter,
blocks=[
    mezzala.blocks.HomeAdvantage(),
    mezzala.blocks.TeamStrength(),
    mezzala.blocks.BaseRate(),
])
model.fit(mh_dict)

DixonColes(adapter=KeyAdapter(home_goals='HG', away_goals='AG', home_team='Home', away_team='Away'), blocks=[TeamStrength(), HomeAdvantage(), BaseRate()]), weight=UniformWeight()

In [172]:
for index, row in df_fixtures_short.iterrows():
    _match_to_predict = {
        'Home': row['home.name'],
        'Away': row['away.name']
    }

    _scorelines = model.predict_one(_match_to_predict)

    _pred = mezzala.scorelines_to_outcomes(_scorelines)
    
    _dict_pred = {
        'home': [row['home.name']],
        'away': [row['away.name']], 
        'game_id': [row['id']],
        'date': [row['status.startDateStr']],
        'prob_away': [_pred.popitem()[1].probability],
        'prob_draw': [_pred.popitem()[1].probability],
        'prob_home': [_pred.popitem()[1].probability],}
    _pred_series = pd.DataFrame(_dict_pred)

    df_predictions = pd.concat([df_predictions, _pred_series], ignore_index=True, axis=0)
    

In [178]:
df_predictions.head()

,home,away,game_id,date,prob_away,prob_draw,prob_home,points_home,points_away
0,Young Boys,FC Zürich,3905393,"Sat, 16 Jul",0.187133,0.212240,0.600627,0.382430,0.119151
1,Winterthur,Basel,3905401,"Sat, 16 Jul",0.593337,0.222316,0.184346,0.122950,0.395726
2,Lugano,Sion,3905400,"Sun, 17 Jul",0.307450,0.274232,0.418318,0.344149,0.252938
3,Servette,St. Gallen,3905399,"Sun, 17 Jul",0.345344,0.259114,0.395542,0.307471,0.268450
4,FC Zürich,Luzern,3905397,"Sat, 23 Jul",0.276367,0.249769,0.473864,0.355070,0.207084


In [189]:
df_predictions['points_home'] = 3 * df_predictions['prob_home'] + 1 * df_predictions['prob_draw']
df_predictions['points_away'] = 3 * df_predictions['prob_away'] + 1 * df_predictions['prob_draw']

In [190]:
df_points = pd.concat([df_predictions[['home','points_home']].groupby('home').sum(),df_predictions[['away','points_away']].groupby('away').sum()], axis=1)
df_points['points_total'] = df_points['points_home'] + df_points['points_away']

In [191]:
df_points.sort_values(by='points_total', ascending=False)

,points_home,points_away,points_total
Basel,38.774490,31.686008,70.460499
Young Boys,37.449768,30.073360,67.523128
FC Zürich,28.995012,21.685565,50.680577
Luzern,27.325709,20.015295,47.341004
Grasshopper,27.145635,19.961145,47.106779
St. Gallen,26.808538,19.475752,46.284290
Sion,25.741267,18.730414,44.471680
Lugano,25.105577,18.189003,43.294580
Servette,24.083953,17.157649,41.241602
Winterthur,22.268198,15.527230,37.795428
